# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
Arxiv has 66 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.10956


extracting tarball to tmp_2404.10956... done.
Retrieving document from  https://arxiv.org/e-print/2404.11523


extracting tarball to tmp_2404.11523...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2404.11523/Main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'properties.tex' from 'tmp_2404.11523/properties.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


P. Mollière  ->  P. Mollière  |  ['P. Mollière']


Found 95 bibliographic references in tmp_2404.11523/Main.bbl.
syntax error in line 193: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.11523-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.11523) | **Fresh view of the hot brown dwarf HD 984 B through high-resolution  spectroscopy**  |
|| J. C. Costes, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2024-04-18*|
|*Comments*| *14 pages, 8 figures*|
|**Abstract**| Context. High-resolution spectroscopy has the potential to drive a better understanding of the atmospheric composition, physics, and dynamics of young exoplanets and brown dwarfs, bringing clear insights into the formation channel of individual objects. Aims. Using the Keck Planet Imager and Characterizer (KPIC; R = 35,000), we aim to characterize a young brown dwarf HD 984 B. By measuring its C/O and 12CO/13CO ratios, we expect to gain new knowledge about its origin by confirming the difference in the formation pathways between brown dwarfs and super-Jupiters. Methods. We analysed the KPIC high-resolution spectrum (2.29-2.49 {\mu}m) of HD 984 B using an atmospheric retrieval framework based on nested sampling and petitRADTRANS, using both clear and cloudy models. Results. Using our best-fit model, we find C/O = 0.50+0.01-0.01 (0.01 is the statistical error) for HD 984 B which agrees with that of its host star within 1{\sigma} (0.40+0.20-0.20). We also retrieve an isotopolog 12CO/13CO ratio of 98+20-25 in its atmosphere, which is similar to that of the Sun. In addition, HD 984 B has a substellar metallicity with [Fe/H] = -0.62+0.02-0.02. Finally, we find that most of the retrieved parameters are independent of our choice of retrieval model. Conclusions. From our measured C/O and 12CO/13CO, the favored formation mechanism of HD 984 B seems to be via gravitational collapse or disk instability and not core accretion, which is a favored formation mechanism for giant exoplanets with m < 13 MJup and semimajor axis between 10 and 100 au. However, with only a few brown dwarfs with a measured 12CO/13CO ratio, similar analyses using high-resolution spectroscopy will become essential in order to determine planet formation processes more precisely. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.10956-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.10956) | **Challenges to Inflation in the Post-Planck Era**  |
|| <mark>J. Liu</mark>, F. Melia |
|*Appeared on*| *2024-04-18*|
|*Comments*| *10 pages, 2 figures. Accepted for publication in ApJ*|
|**Abstract**| Space-based missions studying the cosmic microwave background (CMB) have progressively refined the parameter space in conventional models of inflation shortly ($\sim 10^{-37}$ seconds) after the big bang. While most inflationary scenarios proposed thus far in the context of GR have since been ruled out, the basic idea of inflation may still be tenable, albeit with several unresolved conundrums, such as conflicting initial conditions and inconsistencies with the measured CMB power spectrum. In the new slow-roll inflationary picture, inflation arising in plateau-like potentials requires an initiation beyond the Planck time. This delay may be consistent with the cutoff, $k_{\rm min}$, measured recently in the primordial power spectrum. However, the actual value of $k_{\rm min}$ would imply an initiation time too far beyond the big bang for inflation to solve the horizon problem. In this paper, we also describe several other undesirable consequences of this delay, including an absence of well motivated initial conditions and a significant difficulty providing a viable mechanism for properly quantizing the primordial fluctuations. Nevertheless, many of these inconsistencies may still be avoided if one introduces non-conventional modifications to inflation, such as a brief departure from slow-roll dynamics, possibly due to a dramatic change in the inflationary potential, inflation driven by multiple fields, or a non-minimal coupling to gravity. In addition, some of these difficulties could be mitigated via the use of alternative cosmologies based, e.g., on loop quantum gravity, which replaces the initial big-bang singularity with finite conditions at a bounce-like beginning. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2404.11523.md
    + _build/html/tmp_2404.11523/./Plots/CCF.png
    + _build/html/tmp_2404.11523/./Plots/NS_PTprofile.png
    + _build/html/tmp_2404.11523/./Plots/NS_clear_full.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Nstar}{HD~984}$
$\newcommand{\NTstar}{HIP~115119}$
$\newcommand{\NMstar}{HIP~95771}$
$\newcommand{\Ncomp}{HD~984~B}$
$\newcommand{\as}[1]{\textcolor{blue}{\bf #1}}$
$\newcommand{\pRT}{\texttt{petitRADTRANS}}$
$\newcommand{\rev}[1]{\textcolor{magenta}{\textbf{#1}}\xspace}$</div>



<div id="title">

# Fresh view of the hot brown dwarf $\Ncomp$ through high-resolution spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.11523-b31b1b.svg)](https://arxiv.org/abs/2404.11523)<mark>Appeared on: 2024-04-18</mark> -  _14 pages, 8 figures_

</div>
<div id="authors">

J. C. Costes, et al. -- incl., <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** High-resolution spectroscopy has the potential to drive a better understanding of the atmospheric composition, physics, and dynamics of young exoplanets and brown dwarfs, bringing clear insights into the formation channel of individual objects. Using the Keck Planet Imager and Characterizer (KPIC; $\mathrm{R \approx 35,000}$ ), we aim to characterize a young brown dwarf $\Ncomp$ . By measuring its C/O and $\mathrm{^{12}CO/^{13}CO}$ ratios, we expect to gain new knowledge about its origin by confirming the difference in the formation pathways between brown dwarfs and super-Jupiters. We analysed the KPIC high-resolution spectrum (2.29–2.49 $\mu$ m) of $\Ncomp$ using an atmospheric retrieval framework based on nested sampling and $\pRT$ , using both clear and cloudy models. Using our best-fit model, we find $\rm{C/O}=0.50 \pm 0.01$ (0.01 is the statistical error) for $\Ncomp$ which agrees with that of its host star within 1 $\sigma$ ( $0.40 \pm 0.20$ ). We also retrieve an isotopolog $\mathrm{^{12}CO/^{13}CO}$ ratio of $98^{+20}_{-25}$ in its atmosphere, which is similar to that of the Sun. In addition, $\Ncomp$ has a substellar metallicity with $[$ Fe/H $] = -0.62^{+0.02}_{-0.02}$ . Finally, we find that most of the retrieved parameters are independent of our choice of retrieval model. From our measured C/O and $\mathrm{^{12}CO/^{13}CO}$ , the favored formation mechanism of $\Ncomp$ seems to be via gravitational collapse or disk instability and not core accretion, which is a favored formation mechanism for giant exoplanets with $m<13 \mathrm{M_{Jup}}$ and semimajor axis between 10 and 100 au. However, with only a few brown dwarfs with a measured $\mathrm{^{12}CO/^{13}CO}$ ratio, similar analyses using high-resolution spectroscopy will become essential in order to determine planet formation processes more precisely.

</div>

<div id="div_fig1">

<img src="tmp_2404.11523/./Plots/CCF.png" alt="Fig4" width="100%"/>

**Figure 4. -**  CCFs between the KPIC data (using the three orders from 2.29 to 2.49 $\mu$m) and different models. Top panel: CCFs between the KPIC data and a BT-Settl model (see Section \ref{Preliminary_Analysis}). The CCF for $\Ncomp$ is plotted in blue, and for comparison, we add the CCF of the stellar data in gray. The autocorrelation of the planetary model is also shown in black. The standard deviation of the wings of the CCF were used to estimate its noise and to normalize the CCF. The dashed vertical gray and red lines show the expected and measured RV of the companion ($-23.4 \mathrm{km s^{-1}}$ and $-25.8 \mathrm{km s^{-1}}$, respectively). Bottom panels: CCFs showing the $H_2$O and CO detection from the same KPIC data after using single-molecule templates. (*fig:CCF*)

</div>
<div id="div_fig2">

<img src="tmp_2404.11523/./Plots/NS_PTprofile.png" alt="Fig7" width="100%"/>

**Figure 7. -**  The $\mathrm{P-T}$ profile from our best-fit retrieval (model $\mathrm{MgSiO_{3} + Fe}$). The best-fit profile is shown in black, and 200 random draws from the posterior are shown in blue. The condensation curves for different cloud species ($\mathrm{MgSiO_{3}}$, Fe and $\mathrm{Al_{2}O_{3}}$) are also plotted. In addition, we show multiple Sonora $\mathrm{P-T}$ profiles as dashed lines \citep{Marley2021} with similar properties as $\Ncomp$. Finally, we plot the emission contribution function (in wavelength, top axis) as contours, which quantifies the relative importance of the emission in a given pressure layer to the total at a given wavelength \citep{Molliere2019}. (*fig:PT_profile*)

</div>
<div id="div_fig3">

<img src="tmp_2404.11523/./Plots/NS_clear_full.png" alt="Fig3" width="100%"/>

**Figure 3. -**  Extended corner plot showing the posterior distribution for our clear model retrieval for $\Ncomp$(see Section \ref{clear model}). The titles at the top of each histogram show the median and 1$\sigma$ error. (*fig:Corner_2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.11523"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

405  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
